# Imports

In [102]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon, LinearRing

# Input

In [103]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [104]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [105]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [106]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [107]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.16481474 0.16491063 0.16481474 ... 0.15343697 0.22272627 0.15343697]


[0.37816947 0.37838949 0.37816946 ... 0.35206304 0.51104819 0.35206304]
[0.54769436 0.54772194 0.54769436 ... 0.54441983 0.56429157 0.54441983]
(5999,)


### Multinomial Sampling

In [108]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 1 0 0]


In [109]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[76.96415  ,  7.328478 , 84.60201  ],
       [89.59097  ,  3.5589852, 89.6241   ],
       [88.630486 ,  3.5589852, 91.10313  ],
       [88.09351  ,  3.5589852, 91.80292  ],
       [77.13951  ,  7.328478 , 84.49455  ]], dtype=float32)

# KNN extended graph

In [110]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [111]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11808


# Edge Predictor

In [112]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [113]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [114]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [115]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [116]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 10)	1
  (1, 11)	1
  :	:
  (1498, 1344)	1
  (1498, 1357)	1
  (1498, 1358)	1
  (1498, 1359)	1
  (1498, 1360)	1
  (1498, 1361)	1
  (1498, 1362)	1
  (1498, 1363)	1
  (1498, 1364)	1
  (1498, 1488)	1
  (1498, 1494)	1
  (1499, 941)	1
  (1499, 1004)	1
  (1499, 1006)	1
  (1499, 1010)	1
  (1499, 1012)	1
  (1499, 1335)	1
  (1499, 1336)	1
  (1499, 1337)	1
  (1499, 1354)	1
  (1499, 1355)	1
  (1499, 1356)	1
  (1499, 1357)	1
  (1499, 1376)	1
  (1499, 1494)	1
[[0.07115986 0.07109208 0.07113963 ... 0.06993101 0.07055694 0.07043017]
 [0.98464651 0.06722891 0.06714956 ... 0.06915689 0.0681198  0.06833028]
 [0.97785244 0.54787099 0.06661301 ... 0.06636289 0.06649754 0.06647115]
 ...
 [1.18567148 1.26664233 1.27008728 ... 0.06629379 0.0665576  0.0

In [117]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [118]:
print(A_s)  # symmétrique
print(A_s.shape)

[[1.78958163e-21 1.76084646e-21 1.71313596e-21 ... 2.16191713e-19
  6.11154471e-20 7.25550127e-20]
 [1.76084646e-21 1.73566725e-21 1.69250131e-21 ... 2.33383365e-19
  6.30894783e-20 7.58843870e-20]
 [1.71313596e-21 1.69250131e-21 1.65306433e-21 ... 2.10699069e-19
  5.90386498e-20 7.03887624e-20]
 ...
 [2.16191713e-19 2.33383365e-19 2.10699069e-19 ... 1.88609877e-16
  2.79750739e-17 4.20294415e-17]
 [6.11154471e-20 6.30894783e-20 5.90386498e-20 ... 2.79750739e-17
  4.85237988e-18 6.76738326e-18]
 [7.25550127e-20 7.58843870e-20 7.03887624e-20 ... 4.20294415e-17
  6.76738326e-18 1.03889562e-17]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [119]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

IGRAPH U--- 1500 11808 -- 
+ attr: _nx_name (v)


In [120]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[   5    8   10]
 [1306 1425 1432]
 [ 758  760  764]]
32803


In [121]:
triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[76.538124   6.9390492 84.21874  ]
  [76.37803    6.9390492 84.295105 ]
  [76.21416    6.9390492 84.36298  ]]

 [[78.95516    7.9794145  0.5971703]
  [76.43936    6.5098743  1.3069538]
  [76.608154   6.0434117  2.0139656]]

 [[76.79476   -5.5423326  3.000921 ]
  [77.01562   -6.0434117  2.4910438]
  [78.294556  -7.328478   1.6604918]]]
32803


In [122]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(32803,)
[7.10774729e-22 2.15439633e-18 3.34996713e-16 ... 4.28615465e-16
 4.00165156e-16 9.51578102e-17]


#### Calculate barycenter

In [123]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32803, 3)
[[76.37677    6.9390492 84.292274 ]
 [77.33423    6.844233   1.3060299]
 [77.36831   -6.304741   2.3841522]]


#### KNN Tri

In [124]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [125]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32803, 19, 3)


#### calculate e norm matrix

In [126]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32803, 3)
[[0.17737454 0.3546268  0.17737384]
 [2.9987652  3.3561676  0.8636805 ]]


#### Calculate r

In [127]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32803,)

In [128]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32803, 19)
(32803, 19)


In [129]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32803, 19, 5)


#### Calculate f

In [130]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.0000000000000089
(32803,)
[3.04725609e-05 3.04725609e-05 3.04725609e-05 ... 3.04725609e-05
 3.04725609e-05 3.04725609e-05]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_8064\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [131]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[51.         3.4060426 81.88589  ]
 [51.        -3.710653  97.979416 ]
 [51.        -5.4119864 96.93905  ]]


In [132]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 34
Number of edges: 198


In [133]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [134]:
def torch_d_P_Ps(p_y, x, y):
    """All Tensors in input"""
    # print(p_y.shape, x.shape, y.shape)

    expanded_x1 = x.unsqueeze(1)
    expanded_x2 = y.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)

    min_x = distances.min(dim=1).values
    min_y = distances.min(dim=0)

    first_term = torch.sum(torch.index_select(p_y, 0, min_y.indices) * min_y.values)
    second_term = torch.sum(min_x * p_y)

    return first_term + second_term


torch_d_P_Ps(torch.Tensor(normalized_inclusion_score), torch.Tensor(np.array(graph)), torch.Tensor(np.array(extended_graph)))

tensor(1.0302)

## Probabilistic Surfaces Distance

### d_f_S_Ss

In [135]:
# Creation igraph
igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH U--- 5999 17991 -- 
+ attr: _nx_name (v), index_triangle (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [136]:
# Get finals selected triangles & calculate barycenters
b_hat = np.mean(selected_triangles, axis=1)
print(b_hat.shape)
print("B_HAT : ", b_hat)

(500, 3)
B_HAT :  [[51.         -1.9055322  92.26812   ]
 [59.         -5.8177896  -5.3254504 ]
 [51.         -5.9658127  94.75098   ]
 ...
 [51.          0.52830964 -7.7843075 ]
 [39.          5.578963   -3.1718981 ]
 [55.         -2.5225995   2.5312507 ]]


In [137]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(500, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[ 98.07129628  88.67644254  95.3859296  ...  75.22480879  87.53973334
   71.16053869]
 [ 21.67224123  12.05040932   5.44386534 ...  31.2187126   21.59033187
   34.94321422]
 [101.04438514  91.3773678   97.67061397 ...  77.73769257  90.08957176
   73.67931464]
 ...
 [ 11.86167982  12.20176744  11.07237967 ...  37.54189267  28.80593334
   40.91233812]
 [  5.41137247  18.00421925  21.32604945 ...  43.89578548  38.20483492
   46.35583145]
 [ 18.08642232   3.77021437   8.02424926 ...  27.3101316   20.88280605
   30.38209207]]


In [138]:
minimums_b = np.min(distances, axis=1)
print(minimums_b.shape)
print(minimums_b[:5])

(500,)
[7.0722124  2.97561397 4.16920081 4.35740783 2.15850529]


In [139]:
d_f_S_Ss = np.sum(selected_triangles_indexes * minimums_b)
print(d_f_S_Ss)

44408628.79047325


### d_r_S_Ss

In [140]:
distances = distances
min_d = distances.min(axis=1)
min_d.shape

(500,)

In [141]:
p_y = np.array(final_scores)[selected_triangles_indexes]
p_y.shape

(500,)

In [142]:
first_term = p_y * min_d
first_term.shape

(500,)

In [143]:
number_neigh_selected_barycenters = min(50, len(b))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_selected_barycenters).fit(b).kneighbors(b)

In [144]:
indexes_neigh_selected_barycenters

array([[    0,  3197,  3204, ...,  3209,    67,  2784],
       [    1,  3307,  2916, ...,  3252,    72,    71],
       [    2,  2978,  3427, ...,    81,  3300,  3297],
       ...,
       [11991, 11988, 11165, ..., 11201, 11355, 11365],
       [11992,  2563,  2966, ...,  6500,  6502,  6495],
       [11993,  2965,  2564, ...,  2576, 11359, 11195]], dtype=int64)

In [145]:
xt = b[indexes_neigh_selected_barycenters[:, 1:]].reshape((len(b)*(number_neigh_selected_barycenters-1),3))
print(xt.shape)

y = b_hat
print(y.shape)

(587706, 3)
(500, 3)


In [146]:
distances_barycenter_knn = cdist(xt, y, metric='sqeuclidean')
distances_barycenter_knn.shape

(587706, 500)

In [147]:
pt = final_scores[indexes_neigh_selected_barycenters[:, 1:]].reshape((len(b)*(number_neigh_selected_barycenters-1)))
pt.shape

(587706,)

In [148]:
ptk_norm = pt*distances_barycenter_knn.T
print(ptk_norm.shape)

sum_ptk_norm = np.sum(ptk_norm, axis=1)
print(sum_ptk_norm.shape)

second_term = (1 - p_y) * (1/(number_neigh_selected_barycenters-1)) * sum_ptk_norm
print(second_term.shape)

(500, 587706)
(500,)
(500,)


In [149]:
print(first_term.shape, second_term.shape)

d_r_S_Ss = np.sum(first_term + second_term)
d_r_S_Ss

(500,) (500,)


1007612.5119990171

## Triangle Collision Loss

In [150]:
def compute_lc_le_lo(p_t, m_c_e_o, Fs):
    """
    Compute the collision loss term L_c.

    Parameters:
    - p_t: 1D numpy array containing the probabilities of each triangle (indices)
    - m_c_t: 2D numpy array containing the number of faces penetrated by each triangle
    - Fs: 3D numpy array representing the vertices of triangles

    Returns:
    - L_c: Collision loss term
    """
    assert len(p_t) == len(m_c_e_o), "Input arrays must have the same length"

    penalty_per_triangle = p_t * m_c_e_o

    # Sum the penalties for all selected triangles
    total_penalty = np.sum(penalty_per_triangle)

    # Compute the collision loss term L_c
    L_c_e_o = (1 / len(Fs)) * total_penalty

    return L_c_e_o

# Example usage:
# Replace the arrays below with your actual data
# p_t = selected_triangles_indexes
# m_c_t = numpy array containing the number of faces penetrated by each triangle
# Fs = 3D numpy array representing the vertices of triangles
p_t = selected_triangles_indexes
Fs = triangles  # Given data

In [151]:
number_neigh_barycenters = min(50, len(b_hat))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_barycenters).fit(b_hat).kneighbors(b_hat)

### Lc

In [152]:
mc = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    polygons_others_tri = MultiPolygon([Polygon(others_triangle) for others_triangle in others_triangles]).buffer(0)    # buffer 0 to correct invalid polygons => take the exterior of the shape


    intersection = lines_current_triangle.intersection(polygons_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        mc[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        mc[index_neigh_barycenters[0]] += 1
L_c = compute_lc_le_lo(p_t, mc, Fs)
print("L_c : ", L_c)

L_c :  4225.18776054694


### Le

In [153]:
me = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    lines_others_tri = MultiLineString([LineString([other_tri[0], other_tri[1], other_tri[2], other_tri[0]]) for other_tri in others_triangles])

    intersection = lines_current_triangle.intersection(lines_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        me[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        me[index_neigh_barycenters[0]] += 1
L_e = compute_lc_le_lo(p_t, me, Fs)
print("L_e : ", L_e)

L_e :  24105.658829414708


### Lo

In [154]:
mo = np.zeros((500))

Lo - échantillonnage de 100 points à partir de chaque triangle

In [155]:

def sample_points_from_triangle(t, num_points=100):
    v1, v2, v3 = t
    bary_coords = np.random.rand(num_points, 2)
    sqrt_bary_coords = np.sqrt(bary_coords[:, 0])

    u = sqrt_bary_coords
    v = bary_coords[:, 1]

    """
    La formule spécifique est dérivée de l'expression générale d'interpolation barycentrique 
    sommets A, B et C
    coord barycentriques: u et v
    coord cartésiennes: x,y,z 
    """
    x_coords = (1 - u - v) * v1[0] + u * v2[0] + v * v3[0]
    y_coords = (1 - u - v) * v1[1] + u * v2[1] + v * v3[1]
    z_coords = (1 - u - v) * v1[2] + u * v2[2] + v * v3[2]

    sampled_points = np.column_stack((x_coords, y_coords, z_coords))
    return sampled_points

points100 = sample_points_from_triangle(triangle, num_points=100)
points100
points100.shape

(100, 3)

Les Aires

In [156]:
number_neigh_selected_barycenters = min(50, len(b_hat))
def knnbar(nn):
  _, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=nn).fit(b_hat).kneighbors(b_hat)
  return _, indexes_neigh_selected_barycenters

In [157]:
d, indices = knnbar(number_neigh_selected_barycenters)
indices

array([[  0, 292,  20, ..., 181, 180, 141],
       [  1,   6, 344, ..., 397, 151, 473],
       [  2, 186,  10, ..., 185, 184, 189],
       ...,
       [497, 421, 418, ..., 379, 238, 457],
       [498, 261, 303, ..., 385,  40, 327],
       [499, 264, 126, ..., 404, 338, 320]], dtype=int64)

In [158]:
from scipy.spatial import distance as scipy_distance

def calculate_triangle_area(triangle):
    # Fonction pour calculer l'aire d'un triangle en utilisant la formule de Héron
    side_lengths = [scipy_distance.euclidean(triangle[i], triangle[(i + 1) % 3]) for i in range(3)]
    s = sum(side_lengths) / 2
    return np.sqrt(s * np.prod([s - length for length in side_lengths]))

def penalize_overlapping_triangles(points_list, triangles, k=50):
    # Fonction pour pénaliser les triangles qui se chevauchent
    assignment_results = []
    overlapping_triangles = np.zeros(len(triangles), dtype=int)  # Déclaration en dehors de la boucle

    # ajustement de la valeur maximale de k en fonction du nombre de triangles
    k = min(k, len(triangles))

    # NearestNeighbors pour trouver les k triangles les plus proches pour chaque point
    knn_model = NearestNeighbors(n_neighbors=k).fit(np.vstack(triangles))
    d, indices = knn_model.kneighbors(np.array(points_list)[:, 0, :])

    for i, point_list in enumerate(points_list):
        point = point_list[0]

        # Verif que les indices sont valides
        closest_triangle_indices = indices[i, :k]
        closest_triangle_indices = closest_triangle_indices[closest_triangle_indices < len(triangles)]

        # Accéder aux triangles en utilisant les indices valides
        valid_triangle_indices = []
        for idx in closest_triangle_indices:
            modified_triangle = np.copy(triangles[idx])
            modified_triangle[0] = point

            area_original = calculate_triangle_area(triangles[idx])
            area_modified = calculate_triangle_area(modified_triangle)

            # Vérifier si la somme des aires est proche de l'aire du triangle
            if not np.isclose(area_original, area_modified, rtol=1e-5):
                assignment_results.append((point, triangles[idx]))
                valid_triangle_indices.append(idx)

        # MAJ du overlapping_triangles avec les indices valides
        overlapping_triangles[valid_triangle_indices] += 1

    penalties = overlapping_triangles
    total_penalty = np.sum(penalties)
    Lo = (1 / len(triangles)) * total_penalty

    return Lo


L_o = penalize_overlapping_triangles([points100], triangles, k=50)
print("Pénalité pour les triangles qui se chevauchent :", L_o)


Pénalité pour les triangles qui se chevauchent : 0.0008337502084375521


## Final Loss

lambda_c, lambda_e, lambda_o = 0.01, 0.01, 0.01


Loss_L = prob_chamfer_dist + d_f_S_Ss + d_r_S_Ss + (lambda_c * L_c) + (lambda_e * L_e) + (lambda_o * L_o)
print(Loss_L)